# FACE BLURRING BASED ON AGE DETECTION
Studente: Pierpaolo Gumina<br>
Docente: Prof. Sebastiano Battiato

Si vuole proporre un algoritmo di classificazione in grado di riconoscere uno o
più volti nella scena (face recognition), stimare l’età (age detection) e adottare
tecniche di offuscamento (blurring) con lo scopo di anonimizzare i volti di
individui minorenni.

### Contenuto del progetto:
- [Setup](#Setup)
- [Basato su Immagini](#Age-Detection-on-Image)
- [Basato su Video](#Age-Detection-on-Video)
    - [Basato sull'utilizzo della Webcam](#Age-Detection-on-Live-Webcam-Video)
    - [Basato su un file video locale](#Age-Detection-on-a-Video-File)

In [1]:
# Importazione di librerie utili
from PIL import Image, ImageFilter
import numpy as np
import cv2
import os
from tensorflow.keras.models import load_model
import time
from datetime import datetime

## Setup

In [3]:
# Caricamento del modello di CNN pre-allenato per l'age classification, inoltre si inizializza una lista
# contenente il range di età

model = load_model("age_detect_cnn_model.h5")
age_ranges = ['1-2', '3-9', '10-20', '21-27', '28-45', '46-65', '66-116']

Esistono vari algoritmi pronti all’uso che permettono di realizzare un sistema di
face detection, in particolare per questo progetto è stato utilizzato l’algoritmo
Haar cascades.

In [4]:
# Importazione dell'Haar Cascades classifier in formato XML.

face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [5]:
# Definizione di una funzione per restringere la regione del viso rilevata per una migliorare la previsione nel modello

def shrink_face_roi(x, y, w, h, scale=0.9):
    wh_multiplier = (1-scale)/2
    x_new = int(x + (w * wh_multiplier))
    y_new = int(y + (h * wh_multiplier))
    w_new = int(w * scale)
    h_new = int(h * scale)
    return (x_new, y_new, w_new, h_new)

In [6]:
# Definizione di una funzione per creare la sovrapposizione del testo dell'età prevista sull'immagine.

def create_age_text(img, text, pct_text, x, y, w, h):

    # Definizione del carattere, delle scale e lo spessore.
    fontFace = cv2.FONT_HERSHEY_SIMPLEX
    text_scale = 1.2
    yrsold_scale = 0.7
    pct_text_scale = 0.65

    # Ottenerimento della larghezza, altezza e baseline del testo dell'età.
    (text_width, text_height), text_bsln = cv2.getTextSize(text, fontFace=fontFace, fontScale=text_scale, thickness=2)
    (yrsold_width, yrsold_height), yrsold_bsln = cv2.getTextSize("years old", fontFace=fontFace, fontScale=yrsold_scale, thickness=1)
    (pct_text_width, pct_text_height), pct_text_bsln = cv2.getTextSize(pct_text, fontFace=fontFace, fontScale=pct_text_scale, thickness=1)

    # Calcolo delle coordinate del punto centrale del rettangolo contenente il testo.
    x_center = x + (w/2)
    y_text_center = y + h + 20
    y_yrsold_center = y + h + 48
    y_pct_text_center = y + h + 75

    # Calcolo delle coordinate dell'angolo in basso a sinistra del testo in base alla dimensione del testo e al punto centrale del rettangolo calcolato sopra.
    x_text_org = int(round(x_center - (text_width / 2)))
    y_text_org = int(round(y_text_center + (text_height / 2)))
    x_yrsold_org = int(round(x_center - (yrsold_width / 2)))
    y_yrsold_org = int(round(y_yrsold_center + (yrsold_height / 2)))
    x_pct_text_org = int(round(x_center - (pct_text_width / 2)))
    y_pct_text_org = int(round(y_pct_text_center + (pct_text_height / 2)))

    face_age_background = cv2.rectangle(img, (x-1, y+h), (x+w+1, y+h+94), (0, 100, 0), cv2.FILLED)
    face_age_text = cv2.putText(img, text, org=(x_text_org, y_text_org), fontFace=fontFace, fontScale=text_scale, thickness=2, color=(255, 255, 255), lineType=cv2.LINE_AA)
    yrsold_text = cv2.putText(img, "years old", org=(x_yrsold_org, y_yrsold_org), fontFace=fontFace, fontScale=yrsold_scale, thickness=1, color=(255, 255, 255), lineType=cv2.LINE_AA)
    pct_age_text = cv2.putText(img, pct_text, org=(x_pct_text_org, y_pct_text_org), fontFace=fontFace, fontScale=pct_text_scale, thickness=1, color=(255, 255, 255), lineType=cv2.LINE_AA)

    return (face_age_background, face_age_text, yrsold_text)

In [7]:
def anonymize_face_pixelate(image, blocks=3):
    
    # divide l'immagine di input in blocchi di dimensione NxN 
    (h, w) = image.shape[:2]
    xSteps = np.linspace(0, w, blocks + 1, dtype="int")
    ySteps = np.linspace(0, h, blocks + 1, dtype="int")
    
    # cicla su ciascun blocco in entranbe le direzioni x, y
    for i in range(1, len(ySteps)):
        for j in range(1, len(xSteps)):
            startX = xSteps[j - 1]
            startY = ySteps[i - 1]
            endX = xSteps[j]
            endY = ySteps[i]
            roi = image[startY:endY, startX:endX]
            (B, G, R) = [int(x) for x in cv2.mean(roi)[:3]]
            cv2.rectangle(image, (startX, startY), (endX, endY),
                (B, G, R), -1)
    return image

In [8]:
def anonymize_face_simple(image, factor=3.0):
    # determinare automaticamente la dimensione del kernel di sfocatura in base 
    # alle dimensioni spaziali dell'immagine di input
    (h, w) = image.shape[:2]
    kW = int(w / factor)
    kH = int(h / factor)
    # questo assicura che la larghezza del kernel sia dispari
    if kW % 2 == 0:
        kW -= 1
    # questo assicura che l'altezza del kernel sia dispari
    if kH % 2 == 0:
        kH -= 1
    # applicare una sfocatura gaussiana all'immagine di ingresso utilizzando la
    # dimensione del kernel calcolata precedentemente
    return cv2.GaussianBlur(image, (kW, kH), 0)

In [9]:
# In questa cella si definisce una funzione per trovare i volti in un'immagine per poi 
# classificare ogni volto trovato nelle fasce d'età definite sopra e applicare un filtro di blurring l'addove necessario.

def classify_age(img):

    # Si fa una copia dell'immagine per la sovrapposizione delle età e 
    # un'altra copia in scala di grigi da passare al modello per la classificazione dell'età.
    
    img_copy = np.copy(img)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Rilevare le facce nell'immagine usando la face_cascade caricata precedentemente e 
    # memorizzare le loro coordinate in una lista.
    faces = face_cascade.detectMultiScale(img_copy, scaleFactor=1.2, minNeighbors=6, minSize=(100, 100))

    # Cicla per ciascun volto trovato
    for i, (x, y, w, h) in enumerate(faces):

        # Si disegna un rettangolo intorno alla faccia trovata.
        face_rect = cv2.rectangle(img_copy, (x, y), (x+w, y+h), (0, 100, 0), thickness=2)
        
        # Predire l'età della volto usando il modello caricato precedentemente.
        x2, y2, w2, h2 = shrink_face_roi(x, y, w, h)
        prova=img[y2:y2+h2, x2:x2+w2]
        face_roi = img_gray[y2:y2+h2, x2:x2+w2]
        face_roi = cv2.resize(face_roi, (200, 200))  
        face_roi = face_roi.reshape(-1, 200, 200, 1)
        face_age = age_ranges[np.argmax(model.predict(face_roi))]
        face_age_pct = f"({round(np.max(model.predict(face_roi))*100, 2)}%)"
        
        # Richiamare la funzione definita precedentemente per creare 
        # la sovrapposizione dell'età prevista sull'immagine.
        face_age_background, face_age_text, yrsold_text = create_age_text(img_copy, face_age, face_age_pct, x, y, w, h)
        # se il volto predetto è minorenne si applica il filtro di blurring
        if(face_age in age_ranges[0:3]):
            face=anonymize_face_simple(prova)
            img_copy[y2:y2+h2, x2:x2+w2] = face
    return img_copy

In [10]:
# Difinizione di una funzione per restituire il percorso dell'immagine con il nuovo nome di file.
def new_img_name(org_img_path):
    img_path, img_name_ext = os.path.split(org_img_path)
    img_name, img_ext = os.path.splitext(img_name_ext)

    new_img_name_ext = img_name+"_WITH_AGE"+img_ext
    new_img_path = os.path.join(img_path, new_img_name_ext)

    return new_img_path

In [11]:
# Difinizione di una funzione per restituire il percorso dell'immagine con il nuovo nome di file.
def new_vid_name(org_vid_path):
    vid_path, vid_name_ext = os.path.split(org_vid_path)
    vid_name, vid_ext = os.path.splitext(vid_name_ext)
    new_vid_name_ext = vid_name+"_WITH_AGE"+".mp4"
    new_vid_path = os.path.join(vid_path, new_vid_name_ext)
    return new_vid_path

## Basato su Immagini

In [17]:
# Percorso di salvataggio in locale dell'immagine

my_image = "./img_test/due.jpg"

In [18]:
# Prendere il file nella locazione della cella precedente e passarlo al classificatore

img = cv2.imread(my_image)
age_img = classify_age(img)

try:
    new_my_image = new_img_name(my_image)
    cv2.imwrite(new_my_image, age_img)
    print(f"Saved to {new_my_image}")
except:
    print("Error: Could not save image!")

cv2.imshow("Age Detection on Live Video", age_img)
cv2.waitKey(0);

Saved to ./img_test\due_WITH_AGE.jpg


## Basato su video

La detection basata su video:
1. viene dato in input un video;
2. viene utilizzata la webcam del pc, inferenza in modalità real-time.

### Basato sull'utilizzo della Webcam

Per la rilevazione basata su webcam verrà aperta una nuova finiestra. Premere il tasto "Q" della tastiera per uscire.

In [12]:
cap = cv2.VideoCapture(0)

if (cap.isOpened() == False): 
    print("Unable to read camera feed!")

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
cur_time = datetime.fromtimestamp(time.time()).strftime('%Y%m%d%H%M%S')
out = cv2.VideoWriter(f"Webcam_{cur_time}_WITH_AGE.mp4", fourcc, 18, (frame_width, frame_height))

while(cap.isOpened()):
    
    # Per ogni fotogramma..
    ret, frame = cap.read()
    
    if ret==True:
        
        # Si esegue il classificatore sul fotogramma catturato dalla webcam
        age_img = classify_age(frame)
        
        # Salvataggio del fotogramma sul video in uscita usando l'oggetto VideoWriter definito precedentemente.
        out.write(age_img)

        # Visualizzazione del fotogramma con l'età rilevata.
        cv2.imshow("Age Detection on Live Video", age_img)
        
        # Premere il tasto "Q" per uscire
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    else:
        break
        
# Rilascio dell'ogetto VideoCapture e VideoWriter, e chiusura della finistra.
cap.release()
out.release()
cv2.destroyAllWindows()
print(f"Saved to Webcam_{cur_time}_WITH_AGE.mp4")

Saved to Webcam_20210722121047_WITH_AGE.mp4


### Basato su un file video locale

In [57]:
# Percorso del video

my_video = "./video_test/baby.mp4"

In [60]:
my_video = "./video_test/test.mp4"
cap = cv2.VideoCapture(my_video)

# Controlla se il viedeo può essere aperto
if (cap.isOpened() == False): 
    print("Unable to read video!")

# Ottenere la larghezza e l'altezza del fotogramma video.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Definizione del codec e creazione di un oggetto VideoWriter per salvare il video in uscita nella stessa posizione.
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
new_my_video = new_vid_name(my_video)
out = cv2.VideoWriter(new_my_video, fourcc, 18, (frame_width, frame_height))

while(cap.isOpened()):
    
    # Per ogni frame..
    ret, frame = cap.read()
    
    if ret==True:
        
        # ..si esegue il classificatore sul frame 
        age_img = classify_age(frame)
        
        out.write(age_img)
        
        cv2.imshow("video",age_img)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print(f"Saved to {new_my_video}")

Saved to ./video_test\baby_WITH_AGE.mp4
